<a target="_blank" href="../cluster" style="font-size:20px">All Applications (YARN)</a>

# Стек Hadoop. Практическая работа

## Цель практической работы

Научиться использовать Hadoop MapReduce на практике.

## Что входит в работу

* Загрузка данных в HDFS.
* Получение данных из HDFS.
* Реализация парадигмы MapReduce с применением Hadoop Streaming.

## Формат сдачи

Отправьте в форме сдачи следующие файлы:
- файл с результатом result.json;
- ноутбук с кодом (все команды и функции, которые использовались для решения задач).

# Практическое задание

Будем использовать логи сессий прослушивания музыкальных исполнителей в сервисе Spotify, сокращённую версию.

https://www.aicrowd.com/challenges/spotify-sequential-skip-prediction-challenge/dataset_files

Файл `spotify/log_mini.csv` содержит записи вида `ID сессии, номер в сессии, длинна сессии, id трека, skip_1, skip_2, ...`:
```csv
session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0_00006f66-33e5-4de7-a324-2d18e439fc1e,1,20,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,false,false,false,true,0,0,0,0,0,0,true,16,2018-07-15,true,editorial_playlist,trackdone,trackdone
0_00006f66-33e5-4de7-a324-2d18e439fc1e,2,20,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,false,false,false,true,0,1,0,0,0,0,true,16,2018-07-15,true,editorial_playlist,trackdone,trackdone
```

Вам нужно:
1. **Посчитать для каждого трека количество его прослушиваний. Выведите два самых прослушиваемых трека.**
2. **Вывести долю популярных треков: тех, что имеют больше 100 прослушиваний.**

Для решения задачи:
1. Скопируйте файлы в HDFS.
2. Реализуйте подсчёт прослушиваний отдельным MapReduce, в файлы результата сохраните пары <track_id, listen_count>.
3. С помощью команды `hdfs dfs -cat <YOUR-MAPRED-RESULT/*> | python stream_processor.py` решите три подзадачи:
    1. Подсчитайте количество уникальных треков.
    2. Посчитайте количество треков с количеством прослушиваний больше 20.
    3. Найдите два самых популярных по listen_count.
    
    `stream_processor.py` — скрипт, читающий с потока ввода, необходимо реализовать самостоятельно.
4. Сохраните результат работы скрипта выше в файл `result.json`, формат описан ниже.

Реализуйте решение с использованием Hadoop MapReduce Streaming, для написания mapper и reducer используйте Python.

Решение сохраните в локальный файл `result.json`, где по ключу q1
 запишите ответ на первый вопрос, по ключу q2 — на второй и по ключу q3 — на третий.


## Критерии проверки

1. Корректно реализован алгоритм подсчёта прослушиваний — mapper.py, reducer.py (без сохранения всех данных в память, работа с потоком).
2. mapper.py и reducer.py протестированы локально.
3. Данные ( `spotify/log_mini.csv` ) загружены в HDFS.
4. Корректно запущен процесс Hadoop MapReduce Streaming с использованием mapper.py и reducer.py на данных.
5. Корректно реализован `stream_processor.py` (без сохранения всех данных в память, работа с потоком).
6. Результат записан в файл `result.json` и совпадает с эталонным.

Пример содержимого файла `result.json`:

```json
{
    "q1": ["id1", "id2"],
    "q2": 0.13
}
```

In [1]:
# Пример содержимого файла
! head -n 5 spotify/log_mini.csv

session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0_00006f66-33e5-4de7-a324-2d18e439fc1e,1,20,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,false,false,false,true,0,0,0,0,0,0,true,16,2018-07-15,true,editorial_playlist,trackdone,trackdone
0_00006f66-33e5-4de7-a324-2d18e439fc1e,2,20,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,false,false,false,true,0,1,0,0,0,0,true,16,2018-07-15,true,editorial_playlist,trackdone,trackdone
0_00006f66-33e5-4de7-a324-2d18e439fc1e,3,20,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,false,false,false,true,0,1,0,0,0,0,true,16,2018-07-15,true,editorial_playlist,trackdone,trackdone
0_00006f66-33e5-4de7-a324-2d18e439fc1e,4,20,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,false,false

In [2]:
# Копируем файлы в HDFS
!hdfs dfs -mkdir -p /spotify
!hadoop fs -copyFromLocal spotify/log_mini.csv /spotify/log_mini.csv


copyFromLocal: `/spotify/log_mini.csv': File exists


In [3]:
!hadoop fs -ls /spotify/


Found 1 items
-rw-r--r--   1 jovyan supergroup   28918670 2024-06-15 17:29 /spotify/log_mini.csv


In [4]:
%%writefile mapper.py
# Реализуйте mapper
#!/usr/bin/env python3
import sys

for line in sys.stdin:
    line = line.strip()
        
    fields = line.split(',')
    
    if len(fields) >= 4:
        session_id = fields[0]
        session_position = fields[1]
        session_length = fields[2]
        track_id = fields[3]
        
        print(f"{track_id}\t1")

Overwriting mapper.py


In [5]:
# Протестируйте mapper локально
! head -n 5 spotify/log_mini.csv | python mapper.py

track_id_clean	1
t_0479f24c-27d2-46d6-a00c-7ec928f2b539	1
t_9099cd7b-c238-47b7-9381-f23f2c1d1043	1
t_fc5df5ba-5396-49a7-8b29-35d0d28249e0	1
t_23cff8d6-d874-4b20-83dc-94e450e8aa20	1


In [6]:
%%writefile reducer.py
# Реализуйте reducer
#!/usr/bin/env python3
import sys

current_track_id = None
current_count = 0

for line in sys.stdin:
    line = line.strip()

    track_id, count = line.split('\t', 1)
    try:
        count = int(count)
    except ValueError:
        continue
    
    if current_track_id == track_id:
        current_count += count
    else:
        if current_track_id:
            print(f"{current_track_id}\t{current_count}")
        
        current_track_id = track_id
        current_count = count

if current_track_id:
    print(f"{current_track_id}\t{current_count}")


Overwriting reducer.py


In [7]:
# Протестируйте reducer локально
! python -c "print('\n'.join([f'{x}\t1' for x in (['aa'] * 10 + ['bb'] * 5)]))" | python reducer.py

aa	10
bb	5


In [8]:
!hdfs dfs -ls /spotify/


Found 1 items
-rw-r--r--   1 jovyan supergroup   28918670 2024-06-15 17:29 /spotify/log_mini.csv


In [9]:
# Запустите MapReduce Streaming
! mapred streaming \
  -input /spotify/log_mini.csv \
  -output /track-count \
  -mapper "/opt/conda/bin/python mapper.py" \
  -reducer "/opt/conda/bin/python reducer.py" \
  -file mapper.py \
  -file reducer.py

2024-06-15 17:30:38,037 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper.py, reducer.py] [/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.4.jar] /tmp/streamjob4433980008519002334.jar tmpDir=null
2024-06-15 17:30:38,855 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at /0.0.0.0:8032
2024-06-15 17:30:39,008 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at /0.0.0.0:8032
2024-06-15 17:30:39,221 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/jovyan/.staging/job_1718468542237_0003
2024-06-15 17:30:39,522 INFO mapred.FileInputFormat: Total input files to process : 1
2024-06-15 17:30:39,546 INFO net.NetworkTopology: Adding a new node: /default-rack/127.0.0.1:9866
2024-06-15 17:30:40,414 INFO mapreduce.JobSubmitter: number of splits:2
2024-06-15 17:30:40,652 INFO mapreduce.JobSubmitter: Submitting toke

In [10]:
%%writefile stream_processor.py
# Реализуйте код обработки результата MapReduce
import sys
import json

# Подзадачи:
# 1. Подсчитайте количество уникальных треков:
uniq_tracks_count = 0
# 2. Посчитайте количество треков с количеством прослушиваний больше 20:
popular_tracks_count = 0
# 3. Найдите два самых популярных по listen_count:
top_2_tracks = [None, None]

track_counts = {}

for line in sys.stdin:
    line = line.strip()
    track_id, count = line.split('\t', 1)
    count = int(count)

    track_counts[track_id] = count

# Подсчитываем количество уникальных треков
uniq_tracks_count = len(track_counts)

# Находим треки с количеством прослушиваний больше 20
popular_tracks_count = sum(1 for count in track_counts.values() if count > 20)

# Находим два самых популярных трека
top_2_tracks_sorted = sorted(track_counts.items(), key=lambda x: x[1], reverse=True)[:2]
top_2_tracks = [track_id for track_id, count in top_2_tracks_sorted]

# Сохраняем результаты в файл result.json
data = {
    'q1': uniq_tracks_count,
    'q2': popular_tracks_count,
    'q3': top_2_tracks,
}

with open('result.json', 'w') as f:
    f.write(json.dumps(data, indent=4))


Writing stream_processor.py


In [11]:
# Обработайте данные из HDFS с помощью stream_processor.py
! hdfs dfs -cat /track-count/* | python stream_processor.py

In [12]:
# Выведите содержимое файла result.json
! cat result.json

{
    "q1": 50705,
    "q2": 874,
    "q3": [
        "t_bacf06d3-9185-4183-84ea-ff0db51475ce",
        "t_5718ab08-3a15-4d3f-9e63-42b2f6805e31"
    ]
}